In [347]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import random
import logging
from functools import reduce
from operator import add
import math
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import copy
import time
from random import uniform as randnum

In [348]:
def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1.0 - np.tanh(x)**2

def logistic(x):
    return 1/(1 + np.exp(-x))

def logistic_derivative(x):
    return logistic(x)*(1-logistic(x))

def relu(x):
    return np.maximum(x, 0)

def relu_derivative(x):
    if x <= 0:
        return 0
    return 1


activation_funcs = {
    'relu': relu,
    'tanh': tanh,
    'logistic': logistic
}

der_activation_funcs = {
    'relu': relu_derivative,
    'tanh': tanh_derivative,
    'logistic': logistic_derivative
}

In [571]:
class GANetwork(object):

    def __init__(self, net_architecture, activation):
        
        self.num_layers = len(net_architecture)
        self.net_architecture = net_architecture
        self.biases = [np.random.randn(y, 1) for y in net_architecture[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(net_architecture[:-1], net_architecture[1:])]
        self.activation = activation
        
        self.bias_nitem = sum(net_architecture[1:])
        self.weight_nitem = sum([self.weights[i].size for i in range(self.num_layers-2)])

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = activation_funcs[self.activation](np.dot(w,a)+b)
        return a

    def get_biases_count(self):
        return sum(self.net_architecture[1:])
    
    def get_weights_count(self):
        return sum([self.weights[i].size for i in range(self.num_layers-2)])
    
    def get_error(self, X, y):
        total_error=0
        for i in range(X.shape[0]):
            predicted = self.feedforward(X[i].reshape(-1,1))
            actual = y[i].reshape(-1,1)
            total_error += np.sum(np.power(predicted-actual,2)/2)  # mean-squared error
        return total_error

    def accuracy(self, X, y):
        accuracy = 0
        for i in range(X.shape[0]):
            output = self.feedforward(X[i].reshape(-1,1))
            accuracy += int(np.argmax(output) == np.argmax(y[i]))
        return accuracy / X.shape[0] * 100

class NNGeneticAlgo:

    def __init__(self, n_pops, net_size, activation, mutation_rate, crossover_rate, retain_rate, X, y):
        self.n_pops = n_pops
        self.net_size = net_size
        self.nets = [GANetwork(self.net_size, activation) for i in range(self.n_pops)]
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.retain_rate = retain_rate
        self.X = X[:]
        self.y = y[:]
    
    def get_random_point(self, type):
        net = self.nets[0]
        layer_index, point_index = random.randint(0, net.num_layers-2), 0
        if type == 'weight':
            row = random.randint(0,net.weights[layer_index].shape[0]-1)
            col = random.randint(0,net.weights[layer_index].shape[1]-1)
            point_index = (row, col)
        elif type == 'bias':
            point_index = random.randint(0,net.biases[layer_index].size-1)
        return (layer_index, point_index)

    def get_all_errors(self):
        return [net.get_error(self.X, self.y) for net in self.nets]

    def get_all_accuracies(self):
        return [(net, net.accuracy(self.X, self.y)) for net in self.nets]

    def crossover(self, father, mother):
        # take the father 'genetic'
        net = copy.deepcopy(father)
        # cross-over biases
        for _ in range(self.nets[0].get_biases_count()): 
            if random.uniform(0,1) < self.crossover_rate:
                layer, point = self.get_random_point('bias')
                net.biases[layer][point] = mother.biases[layer][point]
        # cross-over weights
        for _ in range(self.nets[0].get_weights_count()):
            if random.uniform(0,1) < self.crossover_rate:
                layer, point = self.get_random_point('weight')
                net.weights[layer][point] = mother.weights[layer][point]
        return net
        
    def mutation(self, child):
        net = copy.deepcopy(child)
        # mutate biases
        for _ in range(self.nets[0].get_biases_count()):
            if random.uniform(0,1) < self.mutation_rate:
                layer, point = self.get_random_point('bias')
                net.biases[layer][point] += random.uniform(-0.5, 0.5)
        # mutate weights
        for _ in range(self.nets[0].get_weights_count()):
            if random.uniform(0,1) < self.mutation_rate:
                layer, point = self.get_random_point('weight')
                net.weights[layer][point[0], point[1]] += random.uniform(-0.5, 0.5)
        return net

    def evolve(self):
        score_list = list(zip(self.nets, self.get_all_errors()))
        score_list.sort(key=lambda x: x[1])
        score_list = [obj[0] for obj in score_list]
        retain_num = int(self.n_pops*self.retain_rate)
        score_list_top = score_list[:retain_num]
        retain_non_best = int((self.n_pops-retain_num) * self.retain_rate)
        for _ in range(random.randint(0, retain_non_best)):
            score_list_top.append(random.choice(score_list[retain_num:]))

        # breed new childs if current population number less than the population size
        while len(score_list_top) < self.n_pops:
            father = random.choice(score_list_top)
            mother = random.choice(score_list_top)
            if father != mother:
                new_child = self.crossover(father, mother)
                new_child = self.mutation(new_child)
                score_list_top.append(new_child)
        
        self.nets = score_list_top

def trainer(net_arcitecture, activation, X, y, input_shape, num_of_classes, X_test, y_test):
    y = y.reshape(-1, 1)
    enc = OneHotEncoder()
    enc.fit(y)
    y = enc.transform(y).toarray()

    n_generations = 180
    n_populations = 22 
    net_size = copy.deepcopy(net_arcitecture)
    net_size.insert(0,input_shape)
    net_size.append(num_of_classes)

    mutation_rate = 0.2
    crossover_rate = 0.4
    retain_rate = 0.4

    # start our neural-net & optimize it using genetic algorithm
    nnga = NNGeneticAlgo(n_populations, net_size, activation, mutation_rate, crossover_rate , retain_rate, X, y)
    start_time = time.time()
    the_best = 0
 
    print('-' * 80)
    for i in range(n_generations):
        if i % 10 == 0:
            print("Current iteration : {}".format(i+1))
            accuracies = nnga.get_all_accuracies()
            the_best = sorted(accuracies, key=lambda x: x[1], reverse=True)[0]
            print("Current top member's network accuracy: %.2f%%\n" %  the_best[1])
        nnga.evolve()
        
    return the_best[1]

In [572]:
class BPNetwork():

    def __init__(self, num_layers, neurons_in_layer, input_shape, num_of_classes, activation):
        self._num_layers = num_layers + 2  # for the input and output layer
        self._neurons_in_layer = copy.deepcopy(neurons_in_layer)
        self._neurons_in_layer.insert(0,input_shape)
        self._neurons_in_layer.append(num_of_classes)
        self._activation_func = activation
        
        self._weights = []
        for l in range(self._num_layers - 1):
            self._weights.append([])
            for i in range(self._neurons_in_layer[l]):
                self._weights[l].append([])
                for j in range(self._neurons_in_layer[l+1]):
                    self._weights[l][i].append(randnum(-1, 1))

        self._biases = [None]
        for l in range(1, self._num_layers):
            self._biases.append([])
            for i in range(self._neurons_in_layer[l]):
                self._biases[l].append(randnum(-1, 1))

    def FeedForward(self, inputs):
        if len(inputs) != self._neurons_in_layer[0]:
            raise InputSizeError()

        self._activations = [inputs]
        self._weighted_inputs = [None]
        for l in range(1, self._num_layers):
            self._activations.append([])
            self._weighted_inputs.append([])
            for i in range(self._neurons_in_layer[l]):
                weighted_input = 0
                for h in range(self._neurons_in_layer[l-1]):
                    weighted_input += (self._activations[l-1][h] *
                                       self._weights[l-1][h][i])
                self._weighted_inputs[l].append(weighted_input)
                weighted_input += self._biases[l][i]
                self._activations[l].append(activation_funcs[self._activation_func](weighted_input))
        # Return the last layer of activations
        return self._activations[self._num_layers - 1]

    
    def Train(self, inputs_set, expected_set, learning_rate, X_test, y_test):
        expected_set = expected_set.reshape(-1, 1)
        enc = OneHotEncoder()
        enc.fit(expected_set)
        expected_set = enc.transform(expected_set).toarray()
        
        for _ in range(100):
            # Back propogation
            for t in range(len(inputs_set)):
                inputs = inputs_set[t]
                expected = expected_set[t]
                if len(inputs) != self._neurons_in_layer[0]:
                    raise InputSizeError()
                if len(expected) != self._neurons_in_layer[self._num_layers - 1]:
                    raise InputSizeError()
                
                self.FeedForward(inputs)
                errors = [None]
                for l in range(1, self._num_layers):
                    errors.append([])
                # Calculate the errors in the output layer
                for i in range(self._neurons_in_layer[self._num_layers - 1]):
                    error = (self._activations[self._num_layers - 1][i] -
                             expected[i])
                    error *= der_activation_funcs[self._activation_func](self._weighted_inputs[l][i])
                    errors[self._num_layers - 1].append(error)
                # Calculate the errors in the hidden layers
                for l in range(self._num_layers - 2, 0, -1):
                    for i in range(self._neurons_in_layer[l]):
                        error = 0
                        for j in range(self._neurons_in_layer[l+1]):
                            error += self._weights[l][i][j] * errors[l+1][j]
                        error *= der_activation_funcs[self._activation_func](self._weighted_inputs[l][i])
                        errors[l].append(error)
                # Update the biases for each neuron
                for l in range(1, self._num_layers):
                    for i in range(self._neurons_in_layer[l]):
                        self._biases[l][i] -= learning_rate * errors[l][i]
                # Update the weights for each neuron
                for l in range(0, self._num_layers-1):
                    for i in range(self._neurons_in_layer[l]):
                        for j in range(self._neurons_in_layer[l+1]):
                            self._weights[l][i][j] -= (learning_rate *
                                                       self._activations[l][i] *
                                                       errors[l+1][j])
        correctness = 0
        for indx, val in enumerate(X_test):
            res = self.FeedForward(val)
            res = res.index(max(res))
            if res == y_test[indx]:
                correctness += 1
        
        return correctness / len(y_test)



In [573]:
param_alternatives = {
    'num_of_neurons': 20,
    'num_of_layers': 4,
    #'activation': ['relu','identity', 'tanh', 'logistic'],
    'activation': ['relu','tanh', 'logistic']
}

In [574]:
def get_data(dataset_name):
    input_shape = 0
    if(dataset_name == 'iris'):
        iris = datasets.load_iris()
        X = iris.data
        y = iris.target
        nb_classes = 3
        input_shape = 4
    elif(dataset_name == 'digits'):
        digits = datasets.load_digits()
        X = digits.data
        y = digits.target
        nb_classes = 10
        input_shape = 64
    elif(dataset_name == 'wine'):
        digits = datasets.load_wine()
        X = digits.data
        y = digits.target
        nb_classes = 3
        input_shape = 13

    if(input_shape != 0):
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
        return (nb_classes, input_shape, x_train, x_test, y_train, y_test)
    

def train_and_score(network, dataset, trainer_method):
    num_of_layers = len(network['architecture'])
    activation = network['activation']
    architecture = network['architecture']

    nb_classes, input_shape, x_train, x_test, y_train, y_test = get_data(dataset)
    accur = None
    
    if trainer_method == 'GA':
        #Use genetic algorithm
        accur = trainer(architecture, activation, x_train, y_train, input_shape, nb_classes, x_test, y_test)
    elif trainer_method == 'BP':
        # Use back propogation algorithm
        network = BPNetwork(num_of_layers, architecture, input_shape, nb_classes, activation)
        accur = network.Train(x_train, y_train, 1, x_test, y_test)
    elif trainer_method == 'MLP':
        # Use MLPClassifier from sklearn
        model = MLPClassifier(hidden_layer_sizes = architecture, activation = activation, learning_rate_init = 1, max_iter = 400, solver = 'lbfgs').fit(x_train, y_train)
        accur = model.score(x_test, y_test)
    
    return accur

In [575]:

class Network():

    def __init__(self):
        self.accuracy = 0.
        self.network = {}

    def create_random(self):
        self.network['activation'] = random.choice(param_alternatives['activation'])
        num_of_layers = random.randint(1, param_alternatives['num_of_layers'])
        architecture = []
        for _ in range(num_of_layers):
            architecture.append(random.randint(1, param_alternatives['num_of_neurons']))
        self.network['architecture'] = architecture
    
    def train(self, dataset, trainer_method):
        if self.accuracy == 0.:
            self.accuracy = train_and_score(self.network, dataset, trainer_method)

    def print_network(self):
        logging.info(self.network)
        logging.info("Network accuracy: %.2f%%" % (self.accuracy * 100))


In [576]:
class GeneticOperations():
    
    def __init__(self, retain=0.4, random_select=0.1, mutate_chance=0.2):
        self.mutate_chance = mutate_chance
        self.random_select = random_select
        self.retain = retain

    def create_population(self, count):
        pop = []
        for _ in range(0, count):
            network = Network()
            network.create_random()
            pop.append(network)
        return pop

    def fitness(self, network):
        return network.accuracy

    def grade(self, pop):
        summed = reduce(add, (self.fitness(network) for network in pop))
        return summed / float((len(pop)))

    def breed(self, mother, father):
        children = []
        max_layers = param_alternatives['num_of_layers']
        for _ in range(2):
            child = {}
            child['activation'] = random.choice([mother.network['activation'], father.network['activation']])
            child['architecture'] = mother.network['architecture'][:len(mother.network['architecture'])//2] + \
                father.network['architecture'][len(father.network['architecture'])//2:]
            
            network = Network()
            network.network = child

            # randomly mutate some of the children.
            if self.mutate_chance > random.random():
                network = self.mutate(network)
            children.append(network)

        return children

    def mutate(self, network):
        # Choose a random key.
        mutation = random.choice(['architecture', 'activation'])

        # Mutate the selected key.
        if mutation == 'activation':
             network.network['activation'] = random.choice(param_alternatives['activation'])
        else:
            for i in range(2):
                index = random.randint(0,len(network.network['architecture'])-1)
                network.network['architecture'][index] = random.randint(1, param_alternatives['num_of_neurons'])
        return network

    def evolve(self, pop):
        # Get scores for each network.
        graded = [(self.fitness(network), network) for network in pop]
        # Sorts the scores desceding.
        graded = [x[1] for x in sorted(graded, key=lambda x: x[0], reverse=True)]
        # Get the number we want to keep for the next generation.
        retain_length = int(len(graded)*self.retain)
        # The parents are every network we want to keep.
        parents = graded[:retain_length]
        # Randomly keep some of the rest networks
        for individual in graded[retain_length:]:
            if self.random_select > random.random():
                parents.append(individual)

        # fill the rest
#         parents = list(set(parents))
        parents_length = len(parents)
        all_nets = [(net.network['architecture'], net.network['activation']) for net in parents]

        desired_length = len(pop) - parents_length
        children = []
        while len(children) < desired_length:
            male = random.randint(0, parents_length-1)
            female = random.randint(0, parents_length-1)
            if male != female:
                male = parents[male]
                female = parents[female]
                babies = self.breed(male, female)
                for baby in babies:
                    if len(children) < desired_length:
                        baby_net = (baby.network['architecture'], baby.network['activation'])
                        if baby_net not in all_nets:
                            children.append(baby)
                            all_nets.append(baby_net)
        parents.extend(list(children))
        
        print('The generation:')
        print([(net.network['architecture'],net.network['activation'])  for net in parents])
        return parents


In [ ]:
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.DEBUG
)

def train_networks(networks, dataset, trainer_method):
    pbar = tqdm(total=len(networks))
    for network in networks:
        network.train(dataset, trainer_method)
        pbar.update(1)
    pbar.close()

def get_average_accuracy(networks):
    total_accuracy = 0
    for network in networks:
        total_accuracy += network.accuracy

    return total_accuracy / len(networks)

def generate(generations, population, dataset, trainer_method):

    nb_classes, input_shape, x_train, x_test, y_train, y_test = get_data(dataset)
    genetic = GeneticOperations()
    networks = genetic.create_population(population)

    avg_fitness_across_generations = []
    top_fitness_across_generations = []
    
    for i in range(generations):
        logging.info("***Envolving generation %d of %d***" %
                     (i + 1, generations))

        # Train and get accuracy for networks.
        train_networks(networks, dataset, trainer_method)
        
         # Sort our final population.
        networks = sorted(networks, key=lambda x: x.accuracy, reverse=True)

        # Print out the top 5 networks.
        print_networks(networks[:5])

        win_net = networks[0]
        num_of_layers = len(win_net.network['architecture'])
        architecture = win_net.network['architecture']
        activation = win_net.network['activation']

        print('Comparison to MLP from sklearn:')
        model3 = MLPClassifier(hidden_layer_sizes = architecture, activation = activation, learning_rate_init = 1, solver = 'lbfgs').fit(x_train, y_train)
        print('train score:',model3.score(x_train, y_train))
        print('test score:',model3.score(x_test, y_test))
        
        
        # Get the average accuracy for this generation.
        average_accuracy = get_average_accuracy(networks)

        avg_fitness_across_generations.append(average_accuracy * 100)
        top_fitness_across_generations.append(win_net.accuracy * 100)
        
        
        # Print out the average accuracy each generation.
        logging.info("Generation Average: %.2f%%" % (average_accuracy * 100))
        logging.info('-'*80)

        # Evolve, except on the last iteration.
        if i != generations - 1:
            networks = genetic.evolve(networks)
   
    avg_fitness_across_generations.insert(0,0)
    top_fitness_across_generations.insert(0,0)
    
    plt.clf()
    plt.plot(avg_fitness_across_generations, label='avg')
    plt.plot(top_fitness_across_generations, label='top')
    ax = plt.gca()
    ax.set_xlabel('Generation', fontsize=12)
    ax.set_ylabel('Fitness', fontsize=12)
    ax.set_xticks([_ for _ in range(generations + 1)])
    ax.set_xlim(1, 10)
    plt.legend()
    plt.tight_layout()

            
def print_networks(networks):
    for network in networks:
        network.print_network()

def main():
    generations = 10  # Number of times to evole the population.
    population = 20  # Number of networks in each generation.

    generate(generations, population, 'iris', 'BP')

if __name__ == '__main__':
    main()


04/30/2021 12:55:58 PM - INFO - ***Envolving generation 1 of 10***


04/30/2021 01:02:54 PM - INFO - {'activation': 'logistic', 'architecture': [19, 7, 16]}
04/30/2021 01:02:54 PM - INFO - Network accuracy: 97.78%
04/30/2021 01:02:54 PM - INFO - {'activation': 'logistic', 'architecture': [14]}
04/30/2021 01:02:54 PM - INFO - Network accuracy: 97.78%
04/30/2021 01:02:54 PM - INFO - {'activation': 'logistic', 'architecture': [7, 5, 17]}
04/30/2021 01:02:54 PM - INFO - Network accuracy: 97.78%
04/30/2021 01:02:54 PM - INFO - {'activation': 'logistic', 'architecture': [13, 16, 20]}
04/30/2021 01:02:54 PM - INFO - Network accuracy: 95.56%
04/30/2021 01:02:54 PM - INFO - {'activation': 'logistic', 'architecture': [13, 15, 2]}
04/30/2021 01:02:54 PM - INFO - Network accuracy: 88.89%



Comparison to MLP from sklearn:


C:\Users\Or\miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
04/30/2021 01:02:55 PM - INFO - Generation Average: 57.11%
04/30/2021 01:02:55 PM - INFO - --------------------------------------------------------------------------------
04/30/2021 01:02:55 PM - INFO - ***Envolving generation 2 of 10***


train score: 1.0
test score: 0.9333333333333333
The generation:
[([19, 7, 16], 'logistic'), ([14], 'logistic'), ([7, 5, 17], 'logistic'), ([13, 16, 20], 'logistic'), ([13, 15, 2], 'logistic'), ([4, 10], 'logistic'), ([5], 'logistic'), ([6, 3, 6, 20], 'logistic'), ([7, 5], 'logistic'), ([6, 3, 5, 17], 'logistic'), ([13, 10], 'logistic'), ([15, 16], 'logistic'), ([16, 20], 'logistic'), ([13, 7, 16], 'logistic'), ([15, 2], 'logistic'), ([5, 17], 'logistic'), ([6, 20], 'logistic'), ([6, 3, 16, 20], 'logistic'), ([13, 5], 'logistic'), ([14, 20], 'logistic')]


...

train score: 1.0
test score: 0.9333333333333333
The generation:
[([16, 20], 'logistic'), ([19, 7, 16], 'logistic'), ([14], 'logistic'), ([7, 5, 17], 'logistic'), ([13, 16, 20], 'logistic'), ([13, 7, 16], 'logistic'), ([5, 17], 'logistic'), ([13, 10], 'logistic'), ([6, 3, 6, 20], 'logistic'), ([6, 20], 'logistic'), ([13, 6, 20], 'logistic'), ([5, 5, 17], 'logistic'), ([6, 3, 7, 16], 'logistic'), ([16, 20, 17], 'logistic'), ([16, 7, 16], 'logistic'), ([10], 'logistic'), ([13, 14], 'logistic'), ([13, 10, 6], 'logistic'), ([5, 6, 6], 'logistic'), ([16, 6, 20], 'logistic')]


04/30/2021 01:13:35 PM - INFO - {'activation': 'logistic', 'architecture': [16, 20]}
04/30/2021 01:13:35 PM - INFO - Network accuracy: 100.00%
04/30/2021 01:13:35 PM - INFO - {'activation': 'logistic', 'architecture': [19, 7, 16]}
04/30/2021 01:13:35 PM - INFO - Network accuracy: 97.78%
04/30/2021 01:13:35 PM - INFO - {'activation': 'logistic', 'architecture': [14]}
04/30/2021 01:13:35 PM - INFO - Network accuracy: 97.78%
04/30/2021 01:13:35 PM - INFO - {'activation': 'logistic', 'architecture': [7, 5, 17]}
04/30/2021 01:13:35 PM - INFO - Network accuracy: 97.78%
04/30/2021 01:13:35 PM - INFO - {'activation': 'logistic', 'architecture': [16, 20, 17]}
04/30/2021 01:13:35 PM - INFO - Network accuracy: 97.78%



Comparison to MLP from sklearn:


C:\Users\Or\miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
04/30/2021 01:13:35 PM - INFO - Generation Average: 82.67%
04/30/2021 01:13:35 PM - INFO - --------------------------------------------------------------------------------
04/30/2021 01:13:35 PM - INFO - ***Envolving generation 4 of 10***


train score: 1.0
test score: 0.9333333333333333
The generation:
[([16, 20], 'logistic'), ([19, 7, 16], 'logistic'), ([14], 'logistic'), ([7, 5, 17], 'logistic'), ([16, 20, 17], 'logistic'), ([13, 16, 20], 'logistic'), ([13, 7, 16], 'logistic'), ([5, 17], 'logistic'), ([7, 20], 'logistic'), ([5, 7, 16], 'logistic'), ([7, 20, 17], 'logistic'), ([19, 16, 20], 'logistic'), ([5, 20], 'logistic'), ([13, 17], 'logistic'), ([13, 14], 'logistic'), ([13, 5, 17], 'logistic'), ([13, 19, 4], 'logistic'), ([16, 17], 'logistic'), ([17], 'logistic'), ([17], 'relu')]


## תודה רבה!